In [ ]:
from poitousprint import Toflit

client = Toflit()

flows = client.get_flows(year=1789, customs_region="La Rochelle", best_guess_region_prodxpart='1')



In [ ]:
checking = set()
for f in flows :
    product_viz = ''
    if (f['partner_grouping'] == 'Afrique'):
        product_viz = 'traite'
    elif (f['partner_grouping'] == 'Amériques' or f['partner_grouping'] == 'Asie'):
        checking.add(f['product_revolutionempire'])
        product_viz = 'commerce colonial'
    elif (f['product_revolutionempire'] == 'Sel'):
        product_viz = 'sel'
    elif (f['product_revolutionempire'] == 'Eaux-de-vie et liqueurs' or f['product_revolutionempire'] == 'Vins divers'):
        product_viz = 'eau-de-vie et vins divers'
    else :
        product_viz = 'autres produits'
    f['product_viz'] = product_viz
    
    partner_viz = ''
    if (f['partner_grouping'] == 'France'):
        partner_viz = 'Ports francs et petites îles'
    elif (f['partner_grouping'] in ['Flandre et autres états de l\'Empereur', 'Nord', 'Flandres']):
        partner_viz = 'Europe du Nord'
    elif (f['partner_grouping'] == 'Angleterre'):
        partner_viz = 'Angleterre'
    elif (f['partner_grouping'] == 'Afrique'):
        partner_viz = 'Afrique'
    elif (f['partner_grouping'] == 'Amériques' or f['partner_grouping'] == 'Asie'):
        partner_viz = 'Colonies'
    else:
        partner_viz = 'Reste du monde'
        
    f['partner_viz'] = partner_viz


In [ ]:
checking

In [ ]:
import csv

with open('exports.csv', 'w') as csvfile:
    fieldnames = flows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for f in [f for f in flows if f['export_import'] == 'Exports']:
        writer.writerow(f)
        
with open('imports.csv', 'w') as csvfile:
    fieldnames = flows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for f in [f for f in flows if f['export_import'] == 'Imports']:
        writer.writerow(f)